# Lesson 1: classifying cats and dogs

This notebook contains my implementation of the code in week 1 of the Fast AI course.

This is a from-scratch implementation of the steps needed to classify the given image data, aiming to be a bit cleaner than the one provided with the course.

## Setup

In [ ]:
%matplotlib inline

Set the path to the data. This assumes there's a symlink to the data directory in the directory where this notebook is stored.

In [ ]:
#path = "data/dogscats"
path = "data/dogscats/sample/"
model_path = "data/models"

Some imports we'll need in subsequent code. Note that the code in this notebook assumes Python 3.

In [ ]:
import os
import numpy as np
np.set_printoptions(precision=4, linewidth=100)

# TODO: add more only as and when needed

Load (and reload) the utility code we use for later steps.

In [ ]:
from imp import reload
import utils; reload(utils)

## Model setup

In the subsequent steps, we implement the classification using Keras code directly, not using the Vgg16 helper code in the course notebook. First up: import the bits we need.

In [ ]:
from numpy.random import random, permutation
#from scipy import misc, ndimage
#from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import Adam
from keras.preprocessing import image

# SPECIAL MAGIC THAT IF WE DON'T SET IT MAKES THINGS BLOW UP!!!  :-/
K.set_image_dim_ordering('th')

## Model creation

This is a function for adding a VGG16 convolutional block to a model. Each convoluational block adds some zero padding (1 pixel each side), and the actual convolutional layer. This uses convolution with a given number of convolution filters, and 3x3 convolution kernel, and 'relu' activation (a rectified linear unit, i.e. with activation function $f(x)=max(0,x)$, a ramp function).

In [ ]:
def add_conv_block(model, layers, filters):
    for i in range(layers): 
        model.add(ZeroPadding2D((1, 1)))
        model.add(Convolution2D(filters, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

This one adds a fully connected block, with 4096 nodes:

In [ ]:
def add_fully_connected_block(model):
    model.add(Dense(4096, activation = 'relu'))
    model.add(Dropout(0.5))

Do the preprocessing of image data to fit the VGG model.

In [ ]:
# Mean of each channel as provided by VGG researchers
vgg_mean = np.array([123.68, 116.779, 103.939]).reshape((3,1,1))

def vgg_preprocess(x):
    x = x - vgg_mean     # subtract mean
    return x[:, ::-1]    # reverse axis bgr->rgb

This function defines the network architecture:

In [ ]:
def vgg16_model():
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape = (3, 224, 224)))
    
    add_conv_block(model, 2, 64)
    add_conv_block(model, 2, 128)
    add_conv_block(model, 3, 256)
    add_conv_block(model, 3, 512)
    add_conv_block(model, 3, 512)

    model.add(Flatten())
    add_fully_connected_block(model)
    add_fully_connected_block(model)
    model.add(Dense(1000, activation='softmax'))
    return model

Let's create us a model:

In [ ]:
model = vgg16_model()
# model = VGG_16()

Load the weights into the model.

In [ ]:
fpath = get_file('vgg16.h5', model_path + 'vgg16.h5', cache_subdir='models')
model.load_weights(fpath)

Next, we'll grab some images already classified as dogs or cats.

In [ ]:
batch_size = 4

def get_batches(dirname, gen=image.ImageDataGenerator(), shuffle=True, 
                batch_size=batch_size, class_mode='categorical'):
    return gen.flow_from_directory(path + dirname, target_size=(224,224), 
                class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [ ]:
training_batches = get_batches('train', batch_size=batch_size)
validation_batches = get_batches('valid', batch_size=batch_size)

Let's have a look at some of the data:

In [ ]:
imgs, labels = next(training_batches)
utils.plot_images(imgs, titles=labels)

So, the model we have so far can predict imagenet classes. We want to customise this model to predict cats or dogs instead.

In other words, we want to replicate the "fine tune" and"fit" steps in the provided Vgg16 class, but we'll do this directly using the Keras API instead. Then, we'll use the resulting model to predict classifications for the test set.

## Fine tuning

First, pop off the last layer of the model (the 1000-node softmax layer), and add a 2-node one instead.

In [ ]:
model.pop()

Mark all the remaining layers as non-trainable.

In [ ]:
for layer in model.layers: layer.trainable=False

Add a new final layer, 2-node softmax.

In [ ]:
model.add(Dense(2, activation='softmax'))

The model needs to be compiled before we can fit it on data.

In [ ]:
model.compile(optimizer=Adam(lr=0.001),
                loss='categorical_crossentropy', metrics=['accuracy'])

Next, find the class labels we will use (ordered by their label index)

In [ ]:
# We get class labels and indexes from the batches we read earlier
indexes_to_classes = dict((v,k) for k,v in training_batches.class_indices.items())
in_index_order = dict(sorted(indexes_to_classes.items()))
classes = list(in_index_order.values())
print("Class labels: ", classes)

Now we can fit the updated model to our training data.

In [ ]:
model.fit_generator(training_batches, samples_per_epoch=training_batches.nb_sample, nb_epoch=1,
                validation_data=validation_batches, nb_val_samples=validation_batches.nb_sample)